In [36]:
import tkinter as tk
from tkinter import messagebox
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write
import librosa
from tensorflow.keras.models import load_model

In [37]:
model = load_model(r"C:\Users\Saumya Dwivedi\Desktop\guitar_spectrogram_90")

In [38]:
# Chords
chords = ['Dm', 'Am', 'Bb', 'C', 'Bdim', 'Em', 'F', 'G']

In [39]:
# Function to predict the chord
def predict_chord():
    # Record audio for a short duration (e.g., 3 seconds)
    duration = 3
    fs = 22050
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype=np.int16)
    sd.wait()

    # Save the recorded audio
    audio_path = "temp_recording.wav"
    write(audio_path, fs, recording)

    # Load and preprocess the recorded audio
    audio, sr = librosa.load(audio_path, sr=None)
    audio_resampled = librosa.resample(audio, orig_sr=sr, target_sr=22050)
    chromagram = librosa.feature.chroma_stft(y=audio_resampled, sr=22050)
    chromagram = (chromagram - np.min(chromagram)) / (np.max(chromagram) - np.min(chromagram))

    # Pad or truncate the chromagram to match the model input shape
    target_size = (12, 137)
    if chromagram.shape[1] < target_size[1]:
        chromagram = np.pad(chromagram, ((0, 0), (0, target_size[1] - chromagram.shape[1])))
    elif chromagram.shape[1] > target_size[1]:
        chromagram = chromagram[:, :target_size[1]]

    # Reshape to match the model input shape
    chromagram_reshaped = chromagram.reshape((1, 12, 137, 1))

    # Predict chord using the trained model
    predicted_label_index = np.argmax(model.predict(chromagram_reshaped))
    predicted_chord = chords[predicted_label_index]

    # Display the predicted chord
    messagebox.showinfo("Predicted Chord", f"The predicted chord is: {predicted_chord}")


In [40]:
# GUI setup
root = tk.Tk()
root.title("Chord Predictor")

''

In [41]:
# Create and configure widgets
label = tk.Label(root, text="Click the button to record and predict a chord.")
label.pack(pady=20)

predict_button = tk.Button(root, text="Predict Chord", command=predict_chord)
predict_button.pack(pady=10)

exit_button = tk.Button(root, text="Exit", command=root.destroy)
exit_button.pack(pady=10)

In [42]:
# Run the GUI
root.mainloop()